In [5]:
%load_ext autoreload
%autoreload 2

In [33]:
import numpy as np
import SRW
from scipy.sparse import csr_matrix, csc_matrix
import functools

# This toy example is for checking the partial derivatives

In [7]:
edges = np.array([[0,1],[0,2],[1,2],[1,0],[2,0],[2,1]])
features = csc_matrix([[0.9,0.1,1.],[0.1,0.9,1.],[0.5,0.5,1.],[0.9,0.1,1.],[0.1,0.9,1.],[0.5,0.5,1.]])
w0 = np.array([0.7,0.3,0.1])
w1 = np.array([0.7,0.300001,0.1])
rst_prob = 0.5
P_init = mutation_profile = csr_matrix([[1,0,0],[0,1,0],[0,1,1],[1,1,0],[1,1,1]])
nnodes = 3
group_labels = [0, 0, 1, 1, 1]
lam = 0.001

In [8]:
Q0, M_strength0, M_strength_rowSum0, strength_grad0 = SRW.strength_Q_and_gradient(edges, nnodes, features, w0)
Q1, M_strength1, M_strength_rowSum1, strength_grad1 = SRW.strength_Q_and_gradient(edges, nnodes, features, w1)

In [9]:
Q_grad = SRW.Q_gradient_1feature(edges, nnodes, M_strength0, M_strength_rowSum0, strength_grad0.toarray()[:,1]) 
Q_grad.toarray()

array([[ 0.        , -0.07991799,  0.07991799],
       [-0.03630052,  0.        ,  0.03630052],
       [ 0.04380975, -0.04380975,  0.        ]])

In [10]:
(Q1 - Q0).toarray() / 0.000001 # check

array([[ 0.        , -0.07991797,  0.07991797],
       [-0.03630051,  0.        ,  0.03630051],
       [ 0.04380974, -0.04380974,  0.        ]])

In [11]:
P0 = csr_matrix(SRW.iterative_PPR(Q0.toarray(), SRW.renorm(mutation_profile).toarray(), rst_prob))
P1 = csr_matrix(SRW.iterative_PPR(Q1.toarray(), SRW.renorm(mutation_profile).toarray(), rst_prob))

In [12]:
P_grad = SRW.calc_P_grad_pool(edges, nnodes, M_strength0, M_strength_rowSum0, Q0, P0, rst_prob, strength_grad0)
P_grad[1,:,:]

array([[  9.28591017e-05,  -2.24112783e-02,   2.23184192e-02],
       [ -5.49892150e-03,  -9.83871874e-03,   1.53376402e-02],
       [  9.11765236e-04,  -1.32354706e-02,   1.23237053e-02],
       [ -2.70303120e-03,  -1.61249985e-02,   1.88280297e-02],
       [  6.38796524e-04,  -1.62940731e-02,   1.56552766e-02]])

In [13]:
(P1 - P0).toarray() / 0.000001 # check

array([[  9.26206889e-05,  -2.24106175e-02,   2.23179969e-02],
       [ -5.49831308e-03,  -9.83946880e-03,   1.53377818e-02],
       [  9.11855563e-04,  -1.32357492e-02,   1.23238936e-02],
       [ -2.70284617e-03,  -1.61250431e-02,   1.88278893e-02],
       [  6.38777253e-04,  -1.62940386e-02,   1.56552613e-02]])

In [14]:
group2npairs = SRW.count_numbers_per_group(group_labels)

In [15]:
J0 = SRW.obj_func(P0, group_labels, group2npairs, lam, w0)
J1 = SRW.obj_func(P1, group_labels, group2npairs, lam, w1)

In [16]:
J_grad = SRW.obj_func_gradient(P0, P_grad, group_labels, group2npairs, lam, w0)
J_grad

obj_func_gradient finished calculating cost_grad: 10:28:16


array([-0.00818094,  0.01312818,  0.00314724])

In [17]:
(J1 - J0) / 0.000001  # check

0.013126452313994008

In [18]:
J0 = SRW.obj_func(P0, group_labels, group2npairs, lam, w0, 'absolute', 'L1', True)
J1 = SRW.obj_func(P1, group_labels, group2npairs, lam, w1, 'absolute', 'L1', True)

In [19]:
J_grad = SRW.obj_func_gradient(P0, P_grad, group_labels, group2npairs, lam, w0, 'absolute', 'L1', True)
J_grad

obj_func_gradient finished calculating cost_grad: 10:28:18


array([-0.00038486,  0.00299408,  0.00160922])

In [20]:
(J1 - J0) / 0.000001  # check

0.0029933249212632518

# This toy example is for checking the gradient descent functions

In [34]:
edges = [[0,1],[0,2],[0,3],[0,4],[0,5],[1,2],[1,3],[1,4],[1,5],[2,3],[2,4],[2,5],[3,4],[3,5],[4,5],
         [1,0],[2,0],[3,0],[4,0],[5,0],[2,1],[3,1],[4,1],[5,1],[3,2],[4,2],[5,2],[4,3],[5,3],[5,4],
         [0,0],[1,1],[2,2],[3,3],[4,4],[5,5]]
edges = np.array(edges)
features = csc_matrix([[.9,.4,0.,1.],[.9,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.5,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.4,0.,1.],[.9,.6,0.,1.],[.9,.5,0.,1.],
                       [.9,.4,0.,1.],[.9,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.5,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.4,0.,1.],[.9,.6,0.,1.],[.9,.5,0.,1.],
                       [.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.]])
rst_prob = 0.3
P_init = mutation_profile = csr_matrix([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],
                                        [0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]).astype(float)
nnodes = 6
group_labels = [0, 0, 0, 1, 1, 1]
lam = 1e-8
w_init_sd = 0.03
learning_rate = 0.001

In [ ]:
w = SRW.train_SRW_GD(edges, features, nnodes, P_init, rst_prob, group_labels, lam, 
                     w_init_sd, 'squared', 'L1', True, learning_rate=0.2, update_w_func='Nadam')

In [ ]:
w = SRW.train_SRW_GD(edges, features, nnodes, P_init, rst_prob, group_labels, lam, 
                     w_init_sd, 'squared', 'L1', True, learning_rate=2, update_w_func='Nesterov',
                     P_init_validation=P_init, group_labels_validation=group_labels)

In [ ]:
w = SRW.train_SRW_GD(edges, features, nnodes, P_init, rst_prob, group_labels, lam, 
                     w_init_sd, 'squared', 'L1', False, learning_rate=0.1, update_w_func='Adam')

In [36]:
w

array([ 7.85622905, -0.48513362, -2.97608061, -7.24309342])

In [37]:
w, J, d, J_validation = SRW.train_SRW_BFGS(edges, features, nnodes, P_init, rst_prob, group_labels, lam, 
                                           w_init_sd=0.01, loss='squared', norm_type='L1', maximize_diff=False, 
                                           P_init_validation=P_init, group_labels_validation=group_labels)

obj_func_gradient finished calculating cost_grad: 11:43:31
obj_func_gradient finished calculating cost_grad: 11:43:31
obj_func_gradient finished calculating cost_grad: 11:43:31
obj_func_gradient finished calculating cost_grad: 11:43:31
obj_func_gradient finished calculating cost_grad: 11:43:31
obj_func_gradient finished calculating cost_grad: 11:43:31
obj_func_gradient finished calculating cost_grad: 11:43:31
obj_func_gradient finished calculating cost_grad: 11:43:32
obj_func_gradient finished calculating cost_grad: 11:43:32
obj_func_gradient finished calculating cost_grad: 11:43:32
obj_func_gradient finished calculating cost_grad: 11:43:32
obj_func_gradient finished calculating cost_grad: 11:43:32
obj_func_gradient finished calculating cost_grad: 11:43:32
obj_func_gradient finished calculating cost_grad: 11:43:32


In [38]:
w, J, d, J_validation

(array([ 14.42735515,   0.65943753,  -2.59166019, -11.20192713]),
 0.19754669280583287,
 {'funcalls': 14,
  'grad': array([ -2.39742553e-06,   9.48296579e-06,   1.63405493e-07,
           9.79990089e-06]),
  'nit': 13,
  'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'warnflag': 0},
 0.19754640400203285)

### Check the transition matrix Q and propagation scores P

In [39]:
Q = SRW.generate_Q(edges, nnodes, features, w)
Q.toarray()

array([[  5.72923055e-07,   4.96373285e-01,   5.03490896e-01,
          4.21440153e-05,   4.50166003e-05,   4.80849671e-05],
       [  4.98100109e-01,   5.74916185e-07,   5.01763600e-01,
          4.22906293e-05,   4.51732076e-05,   4.82522490e-05],
       [  5.01659442e-01,   4.98205233e-01,   5.70839041e-07,
          4.19907160e-05,   4.48528519e-05,   4.79100575e-05],
       [  4.21443868e-05,   4.21443868e-05,   4.21443868e-05,
          5.72928104e-07,   4.96377660e-01,   5.03495334e-01],
       [  4.51732165e-05,   4.51732165e-05,   4.51732165e-05,
          4.98100206e-01,   5.74916298e-07,   5.01763699e-01],
       [  4.79096275e-05,   4.79096275e-05,   4.79096275e-05,
          5.01654939e-01,   4.98200761e-01,   5.70833916e-07]])

In [40]:
P = SRW.iterative_PPR(Q.toarray(), SRW.renorm(P_init).toarray(), rst_prob)
P

array([[  4.81369662e-01,   2.58030153e-01,   2.60284887e-01,
          1.03572659e-04,   1.04778753e-04,   1.06946993e-04],
       [  2.58927809e-01,   4.80670870e-01,   2.60085778e-01,
          1.03652975e-04,   1.04860003e-04,   1.07029925e-04],
       [  2.59338097e-01,   2.58241323e-01,   4.82105536e-01,
          1.03489145e-04,   1.04694266e-04,   1.06860758e-04],
       [  1.03573572e-04,   1.03294539e-04,   1.03867878e-04,
          4.81370250e-01,   2.58031821e-01,   2.60287193e-01],
       [  1.05143286e-04,   1.04860024e-04,   1.05442052e-04,
          2.58927251e-01,   4.80670880e-01,   2.60086424e-01],
       [  1.06557015e-04,   1.06269945e-04,   1.06859799e-04,
          2.59335782e-01,   2.58239595e-01,   4.82104937e-01]])

In [41]:
group2npairs = SRW.count_numbers_per_group(group_labels)

In [42]:
J = SRW.obj_func(P, group_labels, group2npairs, lam, w, loss='squared', norm_type='L1', maximize_diff=False)
J

0.19754669280583287

### Compare with unweighted random walk

In [43]:
Qu = SRW.generate_Q_unweighted(edges, nnodes)
Qu.toarray()

array([[ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667]])

In [44]:
Pu = SRW.iterative_PPR(Qu.toarray(), SRW.renorm(P_init).toarray(), rst_prob)
Pu

array([[ 0.41666667,  0.11666667,  0.11666667,  0.11666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.41666667,  0.11666667,  0.11666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.41666667,  0.11666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.11666667,  0.41666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.11666667,  0.11666667,  0.41666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.11666667,  0.11666667,  0.11666667,
         0.41666667]])

In [45]:
Ju = SRW.obj_func(Pu, group_labels, group2npairs, lam, [0.,0.,0.,0.], loss='squared', norm_type='L1', maximize_diff=False)
Ju

0.36000000000000004

# Scratch